### Spark Sentence Detector

In [ ]:
#!pip install sparknlp

In [ ]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from pyspark.ml import Pipeline
import pandas as pd
import re
import nltk
from nltk import word_tokenize
#pd.set_option("display.max_colwidth", 0)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
builder = SparkSession.builder\
        .appName("Spark NLP Licensed")\
        .master("local[*]")\
        .config("spark.driver.memory", "24G")\
        .config("spark.driver.maxResultSize", "2048GB")\
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
        .config("spark.kryoserializer.buffer.max", "2000M")\
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")

spark = builder.getOrCreate()

In [ ]:
spark

In [ ]:
#textFiles = spark.sparkContext.wholeTextFiles("oncology_notes/*")

#df = textFiles.toDF(schema=['path','text'])
#df.show()

In [ ]:
# custom character alphabet 
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüßé0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Deutch
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî0123456789\.\?\{\}\[\]\(\),:;!'\"\$`]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓ۱عٮێڔ۶ؚۧۜڤۏ٤إٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ٣؍ي٧؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨ۹ٹڵڪظٶۭ١۔ڭەڨحٕ؎ٺڷٰ۪۫۲ڻڥۛڑڟټآڡغګ؊ّ٢ٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفر۷ڇ٭چڝ۴بۈٽڕۡھ۸٨ةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻش٥ڍ؇۵ك۳خ\.\?\{\}\[\]\(\)\"\$\ا0123456789,':;\"`]+", re.IGNORECASE|re.UNICODE) # Arabic

In [ ]:
part_number = 1
text_name = "tr"
custom_alp = tr
sentence_tokenize_name = f"Spark_Sentence_Tokenize{part_number}"

In [ ]:
df = spark.read.text(f'/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Pyspark/{text_name}{part_number}.txt').toDF('text') # text data must be without text pre process

In [ ]:
df.select("text").head(1)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df = df.repartition(200)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentencerDL = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

model = PipelineModel(stages=[documenter, sentencerDL])

In [ ]:
result = model.transform(df)

In [ ]:
result.show()

In [ ]:
result.printSchema()

In [ ]:
pd_res = result.select("sentences.result").toPandas()
pd_res

In [ ]:
pd_res.to_csv(f"{sentence_tokenize_name}.csv", index=False)

#### Clean DataFrame Sentence

In [ ]:
#!pip install modin[ray]

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

In [ ]:
import modin.pandas as pd

In [ ]:
df2 = pd.read_csv(f"{sentence_tokenize_name}.csv")
df2

In [ ]:
## will test 
#def clean_text(text):
#    apostro = re.compile(re.escape("\'"), re.IGNORECASE)
#    text2 = apostro.sub(" \'", text)
#    text3 = re.findall(custom_alp, text2)
#    text4 = " ".join(text3)
#    clean_brackets = text4.strip("[]")
#    clean_brackets_data = re.sub('[\(\[\{].*?[\)\]\}]', '', clean_brackets, re.UNICODE)
#    clean_punc = re.sub(r'[^\w\s]','',clean_brackets_data, re.UNICODE)
#    text_clean_list = re.findall(r'\w+', clean_punc, re.UNICODE)
#    #alpha_num = [w for w in text_clean_list if w.isalnum()]
#    text_result = (" ".join(text_clean_list)).lower()
#    return text_result 

In [ ]:
def clean_text(text):
    clean_brackets = text.strip("[]")
    clean_brackets_data = re.sub('[\(\[\{].*?[\)\]\}]', '', clean_brackets, re.UNICODE)
    clean_punc = re.sub(r'[^\w\s]','',clean_brackets_data, re.UNICODE)
    text_clean_list = re.findall(r'\w+', clean_punc, re.UNICODE)
    #alpha_num = [w for w in text_clean_list if w.isalnum()]
    text_result = (" ".join(text_clean_list)).lower()
    return text_result 

In [ ]:
sent_list = []
for x in df2.result:
    x = clean_text(x)
    sent_list.append(x)  

In [ ]:
df_clean = pd.DataFrame(sent_list)
df_clean.head()

In [ ]:
df_clean = df_clean.rename(columns={0:"sentence"})
df_clean

In [ ]:
df_clean.drop(df_clean[df_clean.sentence == ""].index, inplace=True)

In [ ]:
sent_count = df_clean.sentence.value_counts().sort_values(ascending=False)
#sent_count

In [ ]:
sent_count_df = pd.DataFrame(sent_count).reset_index()
sent_count_df.rename(columns={"index":"sentence", "sentence":"frequency"}, inplace=True)

In [ ]:
total_frequency = sent_count_df.frequency.sum()

In [ ]:
sent_count_df["ratio"] = (sent_count_df.frequency/total_frequency)*100

In [ ]:
def sentence_lenght(sentence):
    word_var2 = re.findall(r'\w+', sentence, re.UNICODE)
    if len(word_var2) <= 10:
        return sentence
    else:
        return "sentence is bigger than ten word"

In [ ]:
sent_count_df.sentence = sent_count_df.sentence.apply(sentence_lenght)

In [ ]:
sent_count_df.drop(sent_count_df[sent_count_df.sentence == "sentence is bigger than ten word"].index, axis=0, inplace=True)
sent_count_df.reset_index(drop=True, inplace=True)

In [ ]:
df_result = sent_count_df[sent_count_df.frequency>1]

In [ ]:
df_result.to_csv(f"Clean_{sentence_tokenize_name}.csv", index=False)